# Importing Packages and creating universally useful tools

In [1]:
import numpy as np
import pandas as pd
import re
import requests
from collections import Counter
import bs4 as bs
import unicodedata
mdict = {"January":"1", "February":"2", "March":"3", "April":"4", "May":"5", "June":"6", "July":"7",\
         "August":"8", "September":"9", "October":"10", "November":"11", "December":"12"}
import us
STATES_DICT = us.states.mapping('name', 'abbr')

def strip_accents(text):
    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    return str(text)
def stripper(begins):
    begins = [strip_accents(boo) for boo in begins]
    begins = [re.sub(r"%C3%BA","u", u) for u in begins]
    begins = [re.sub(r"%C3%A9","e", e) for e in begins]
    begins = [re.sub(r"%C3%AD","i", i) for i in begins]
    begins = [re.sub(r"%C3%A1","a", a) for a in begins]
    begins = [re.sub("%27", "'", nra) for nra in begins]
    #begins = [re.sub(" [A-Z]\. ", " ", nra) for nra in begins]
    return(begins)




## It begins

In [77]:
section = pd.read_html("https://en.wikipedia.org/wiki/List_of_current_United_States_governors")[0]
section = section.iloc[2:len(section),:]
section.columns = ["State", "Chamber", "Name", "DeleteA", "Party", "DeleteB", "Date", "End", "DeleteC"]
del(section["DeleteA"])
del(section["DeleteB"])
del(section["DeleteC"])
section["Chamber"] = ["governor" for x in range(len(section))]
section["State"] = section["State"].str.lower()
section["Party"] = section["Party"].str.lower()
section["Name"] = section["Name"].str.lower()
section["End"] = section["End"].str.replace("[^0-9]+","")
section["Party"] = pd.Series([x[0] for x in section["Party"].tolist()])
section = section.reset_index(drop = True)
dates = section["Date"].tolist()
year = [re.findall("\d\d\d\d",x)[0] for x in dates]
day = [re.sub(",","",re.findall("\d+,",x)[0]) for x in dates]
month = [mdict[re.findall("^[A-Z][a-z]+",x)[0]] for x in dates]
df = pd.DataFrame({'year': year,
                       'month': month,
                       'day': day})
date = pd.to_datetime(df)
section["Date"] = date
#section = section.sort_values(by = "Date")
section

,State,Chamber,Name,Party,Date,End
0,alabama,governor,kay ivey,r,2017-04-10,2019
1,alaska,governor,bill walker,r,2014-12-01,2018
2,arizona,governor,doug ducey,d,2015-01-05,2019
3,arkansas,governor,asa hutchinson,d,2015-01-13,2019
4,california,governor,jerry brown,d,2011-01-03,2019
5,colorado,governor,john hickenlooper,d,2011-01-11,2019
6,connecticut,governor,dannel malloy,r,2011-01-05,2019
7,delaware,governor,john carney,r,2017-01-17,2021
8,florida,governor,rick scott,d,2011-01-04,2019
9,georgia,governor,nathan deal,r,2011-01-10,2019


In [87]:
section = pd.read_html("https://en.wikipedia.org/wiki/List_of_current_United_States_governors")[0]
section = section.iloc[2:len(section),:]
section.columns = ["State", "Chamber", "Name", "DeleteA", "Party", "DeleteB", "Date", "End", "DeleteC"]
del(section["DeleteA"])
del(section["DeleteB"])
del(section["DeleteC"])
section["Chamber"] = ["governor" for x in range(len(section))]
section["State"] = section["State"].str.lower()
section["Party"] = section["Party"].str.lower()
section["Name"] = section["Name"].str.lower()
section["End"] = section["End"].str.replace("[^0-9]+","")
dates = section["Date"].tolist()
year = [re.findall("\d\d\d\d",x)[0] for x in dates]
day = [re.sub(",","",re.findall("\d+,",x)[0]) for x in dates]
month = [mdict[re.findall("^[A-Z][a-z]+",x)[0]] for x in dates]
df = pd.DataFrame({'year': year,
                       'month': month,
                       'day': day})
date = pd.to_datetime(df)
section["Date"] = date
#section = section.sort_values(by = "Date")
#section = section.reset_index(drop = True)
#section

In [86]:
#section["Party"] = pd.Series([x[0] for x in section["Party"].tolist()])
#pd.Series([x[0] for x in section["Party"].tolist()])

In [99]:
section = pd.read_html("https://en.wikipedia.org/wiki/List_of_current_United_States_governors")[0]
section = section.iloc[2:len(section),:]
section.columns = ["State", "Chamber", "Name", "DeleteA", "Party", "DeleteB", "Date", "End", "DeleteC"]
del(section["DeleteA"])
del(section["DeleteB"])
del(section["DeleteC"])
section["Chamber"] = ["governor" for x in range(len(section))]
section["State"] = section["State"].str.lower()
section["Party"] = section["Party"].str.lower()
section["Name"] = section["Name"].str.lower()
section["End"] = [re.findall("\d\d\d\d",x)[0] for x in section["End"]]
dates = section["Date"].tolist()
year = [re.findall("\d\d\d\d",x)[0] for x in dates]
day = [re.sub(",","",re.findall("\d+,",x)[0]) for x in dates]
month = [mdict[re.findall("^[A-Z][a-z]+",x)[0]] for x in dates]
df = pd.DataFrame({'year': year,
                       'month': month,
                       'day': day})
date = pd.to_datetime(df)
section = section.reset_index(drop = True)
section["Date"] = date
section

,State,Chamber,Name,Party,Date,End
0,alabama,governor,kay ivey,republican,2017-04-10,2019
1,alaska,governor,bill walker,independent,2014-12-01,2018
2,arizona,governor,doug ducey,republican,2015-01-05,2019
3,arkansas,governor,asa hutchinson,republican,2015-01-13,2019
4,california,governor,jerry brown,democratic,2011-01-03,2019
5,colorado,governor,john hickenlooper,democratic,2011-01-11,2019
6,connecticut,governor,dannel malloy,democratic,2011-01-05,2019
7,delaware,governor,john carney,democratic,2017-01-17,2021
8,florida,governor,rick scott,republican,2011-01-04,2019
9,georgia,governor,nathan deal,republican,2011-01-10,2019


In [98]:
date

0    2017-04-10
1    2014-12-01
2    2015-01-05
3    2015-01-13
4    2011-01-03
5    2011-01-11
6    2011-01-05
7    2017-01-17
8    2011-01-04
9    2011-01-10
10   2014-12-01
11   2007-01-01
12   2015-01-12
13   2017-01-09
14   2017-05-24
15   2018-01-31
16   2015-12-08
17   2016-01-11
18   2011-01-05
19   2015-01-21
20   2015-01-08
21   2011-01-01
22   2011-01-03
23   2012-01-10
24   2018-06-01
25   2013-01-07
26   2015-01-08
27   2011-01-03
28   2017-01-05
29   2018-01-16
30   2011-01-01
31   2011-01-01
32   2017-01-01
33   2016-12-15
34   2011-01-10
35   2011-01-10
36   2015-02-18
37   2015-01-20
38   2015-01-06
39   2017-01-24
40   2011-01-08
41   2011-01-15
42   2015-01-20
43   2009-08-11
44   2017-01-05
45   2018-01-13
46   2013-01-16
47   2017-01-16
48   2011-01-03
49   2011-01-03
dtype: datetime64[ns]

In [100]:
pd.Series([x[0] for x in section["Party"].tolist()])

0     r
1     i
2     r
3     r
4     d
5     d
6     d
7     d
8     r
9     r
10    d
11    r
12    r
13    r
14    r
15    r
16    r
17    d
18    r
19    r
20    r
21    r
22    d
23    r
24    r
25    d
26    r
27    r
28    r
29    d
30    r
31    d
32    d
33    r
34    r
35    r
36    d
37    d
38    d
39    r
40    r
41    r
42    r
43    r
44    r
45    d
46    d
47    r
48    r
49    r
dtype: object